### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Optimal Skill-Based Worker Assignment

#### Noah Rhodes (nrhodes@wisc.edu), Alliot Nagle (acnagle@wisc.edu), and Liam Marshall (limarshall@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

An important aspect of project management is assigning employees to tasks, and doing so in an efficient manner.
Several aspects can be considered such as decreasing worker boredom _[1]_, maximizing productivity, or increasing robustness to workers not showing up.

Our project is focusing on minimizing the time to complete a project, given a set of workers and tasks.
Our unique twist is to account for how repeating a task can lead to decreasing time required to complete it, which we call the worker’s _learning model_.

The goal of the project is to find an optimal schedule for assigning workers with specific set of skills to a set of tasks.
Each task has a defined amount of work that needs to be done in order to complete a task, and the amount of time a worker spends on completing a task depends on his or her skill set.
In addition, when a worker completes a task, they become more skilled at performing that task, completing it more quickly in the future.
An optimal schedule could minimize the length of a project timeline by assigning workers to tasks that match the skills they have, or it might assign a low skill worker to a job several times to in order for them to learn the skill and be highly productive in the long run.  

The inspiration for this project comes from work with Engineers Without Borders.
As designs for a bridge, a microgrid, or a water system are finalized, a construction schedule must be planned.
Every household in the community contributes a person to the construction, which means access to a large amount of labor; however, these workers are not typically highly skilled in the type of required work.
However, at low skill levels, repeated work can lead to a rapidly increasing productivity **[cite?]**.
A tool to better estimate how worker productivity may change over the course of a project would dramatically help in the creation of feasible project timelines. 

In large projects, the number of tasks often significantly outnumbers the workers and we must decide how to assign our workers.
Do we have workers that specialize in a task and complete it quickly, or instead have workers who can generalize and complete a variety of tasks more slowly.  How do we assign our workers to a project timeline to optimize the completion time?

We develop a model to find an optimal schedule for worker assignment based on the amount of work each task requires and the skill level of each worker.
This model can be useful for a wide array of applications.
For example, developing a construction timeline and determining what combination of worker assignments will allow a project to be completed quickly.
An additional use may be to decide if it will be more productive to have a worker learn on the job or instead give a worker training where they stop producing work but may return with a higher skill level. 

We data for our model, using very broad assumptions/estimates.
Our goal is to prove that that our model can be viable for a variety of assignment problems, rather focus on a specific worker dataset.
We will create worker skill levels and learning speeds from gaussian distributions **[update?]** restricted to a sensible range of values.

Depending on the variety of types of tasks in a project, the quantity of those tasks, the number of workers, and any initial skills the workers have, we can expect a different optimal assignment of tasks to workers.
This is leads to a question of whether workers should generalize or specialize in a skill, or how short-term optimal worker assignment compares to long-term optimal assignment.
Our project hopes to provide an answer to these questions and their dependency on initial conditions. 

----

[1]: Bhadury, J., and Z. Radovilsky. "Job rotation using the multi-period assignment model." _International Journal of Production Research_ 44.20 (2006): 4431-4444.

## 2. Mathematical model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$
\begin{bmatrix}
  1 & 2 \\
  3 & 4
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix} =
\begin{bmatrix} 5 \\ 6 \end{bmatrix}
$$

And here is an example of an optimization problem in standard form:

$$
\begin{aligned}
\underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
\text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
& h_j(x) = 0 && j=1,\dots,r
\end{aligned}
$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
# this is a code block
using JuMP, Clp
m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.